# 1. Logistic Regression

In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

data_file_1 = Path(os.path.abspath("")) / "data" / "ex2data1.txt"
data_set_1 = pd.read_csv(data_file_1, header=None, names=["Score_1", "Score_2", "Admitted"])
# print(data_set_1)
# print(data_set_1.head())
# print(data_set_1.describe())

x = np.array(data_set_1.iloc[:, 0:2])
y = np.array(data_set_1.iloc[:, 2:])
m, n = x.shape
print(m, n)
print(x)
print(y)

## 1.1 Visualizing the data

In [ ]:
# init empty array with dimensions
admitted_list = np.empty((0, 3))
non_admitted_list = np.empty((0, 3))

for record in np.array(data_set_1):
    if record[2] == 1:
        admitted_list = np.append(admitted_list, [record], axis=0)
    else:
        non_admitted_list = np.append(non_admitted_list, [record], axis=0)

# simple way to init
# admitted_list = np.array([record for record in np.array(data_set_1) if record[2] == 1])
# non_admitted_list = np.array([record for record in np.array(data_set_1) if record[2] == 0])

# print(admitted_list)
# print(not_admitted_list)

plt.plot(admitted_list[:, 0], admitted_list[:, 1], 'go', label="Admitted")
plt.plot(non_admitted_list[:, 0], non_admitted_list[:, 1], 'ro', label="Non-admitted")
plt.xlabel("Score_1")
plt.ylabel("Score_2")
plt.legend(loc="upper right")
plt.show()


## 1.2 Implementation
### 1.2.1 Sigmoid function

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# draw sigmoid
sigmoid_x = np.linspace(-10, 10)
sigmoid_y = sigmoid(sigmoid_x)
plt.plot(sigmoid_x, sigmoid_y)
plt.xlabel("z")
plt.ylabel("sigmoid(z)")
plt.show()

### 1.2.2 Cost function and gradient

In [ ]:
X = np.insert(x, 0, np.ones(m), axis=1)
# print(X)
init_theta = np.zeros((n + 1, 1))
# print(init_theta)

def cost(theta, X, y):
    h = sigmoid(X @ theta)
    m = X.shape[0]
    j = 1 / m * (-y.T @ np.log(h) - (1 - y).T @ np.log(1 - h))
    return j.item()

def gradient(theta, X, y):
    m = X.shape[0]
    gradient = 1 / m * X.T @ (sigmoid(X @ theta) - y)
    return gradient

# cost and gradient when theta = [[0], [0], [0]]
print(cost(init_theta, X, y))
print(gradient(init_theta, X, y))

# cost and gradient when theta = [[-24], [0.2], [0.2]]
print(cost(np.matrix([[-24], [0.2], [0.2]]), X, y))
print(gradient(np.matrix([[-24], [0.2], [0.2]]), X, y))

### 1.2.3 Learning parameters using library

In [ ]:
# use TNC algorithm to find best theta
# notice that y need to be 1-d array when using this method
res = opt.minimize(fun = cost, x0 = init_theta, args = (X, y.flatten()), method = 'TNC', jac = gradient)
learned_theta = res.x
print(res)
print(learned_theta)

# draw decision boundary
res_x1 = np.linspace(X[:, 1].min(), X[:, 1].max(), 100)
res_x2 = (0 - learned_theta[0] - learned_theta[1] * res_x1) / learned_theta[2]
print(res_x1)
print(res_x2)

plt.plot(admitted_list[:, 0], admitted_list[:, 1], 'go', label="Admitted")
plt.plot(non_admitted_list[:, 0], non_admitted_list[:, 1], 'ro', label="Non-admitted")
plt.plot(res_x1, res_x2, "-")
plt.xlabel("Score_1")
plt.ylabel("Score_2")
plt.legend(loc="upper right")
plt.show()

### 1.2.4 Evaluating logistic regression

In [ ]:
def hypothesis(theta, x):
    return sigmoid(theta.T @ x).item()

print("probability: {}".format(hypothesis(learned_theta, np.array([[1], [45], [85]]))))

def predict(theta, X):
    p = sigmoid(X @ theta)
    res = np.array([1 if record >= 0.5 else 0 for record in p])
    return res

predict_res = predict(learned_theta, X)
predict_accuracy = np.sum([1 if predict_res[i] == y[i][0] else 0 for i in range(len(y))]) / len(y)
print("accuracy on training data: {}".format(predict_accuracy))